# La loi du $\chi^2$ et son test

In [2]:
import numpy as np
from scipy.stats import chi2

Sur ces lois statistiques de [scipy](https://dataforeveryone.medium.com/pdf-pmf-et-cdf-en-1-minute-bd3da21f7d70) il existe toujours plusieurs sous commandes qui [sont toujours](https://dataforeveryone.medium.com/pdf-pmf-et-cdf-en-1-minute-bd3da21f7d70):
- `pdf`: *probability density function*, densité de probabilité (pour les distributions continues).
- `cdf`: *cumulative distribtion function*, probalité cumulée (intégrale de `pdf`).
- `pmf`: *probability mass function*, probabilité d'obtenir une valeur pour une distribution discrète.
- `ppf`: *percent point function*, inverse de la fonction `cdf` donnant le pourcentage correspondant à l'intégrale de la distribution.
- `sf`: *survival function*, fonction de survie `1-cdf`.
- `isf: *inverse survival function*, inverse de la fonction de survie `sf`.
- `stats`: les moments: moyenne (`'v'`), la variance (`'v'`), la distorsion (`'s'`, *skew* en anglais), l'aplatissement (`'k'` *kurtosis* en anglais) qui sont donnés par le paramètre `moments=`.
- `moment([order])`: moment non centré correspondant aux ordres données en paramètre.

Les lois de distribution continues descendent toutes de l'objet [rv_continuous](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_continuous.html#scipy.stats.rv_continuous) et les distributions discrètes de l'objet [rv_discrete](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rv_discrete.html#scipy.stats.rv_discrete).
  

Voici la façon la plus simple d'utiliser le test du $\chi^2$. Il s'agit de vérifier si un jeu de données ayant un certain degré de liberté `n` est constitués ou non de `n` variables indépendantes.

In [3]:
confidence = 0.95
freedom = 5

threshold = chi2.ppf(confidence, freedom)
print(f'Pour la distribution T, avec {freedom:.0f} degrés de liberté, P(T < {threshold:.2f}) = {confidence*100:.0f}%')
print(f'Le seuil à ne pas dépasser pour que l\'hypothèse H0 soit vraie à {confidence*100:.0f}% est {threshold:.2f}' )

Pour la distribution T, avec 5 degrés de liberté, P(T < 11.07) = 95%
Le seuil à ne pas dépasser pour que l'hypothèse H0 soit vraie à 95% est 11.07


## Vérifier si un dé est truqué

Cet exemple vient de la page [Wikipédia](https://fr.wikipedia.org/wiki/Test_du_%CF%87%C2%B2). Le test consiste à lancer 600 fois le dé. S'il n'est pas truqué, nous devrions normalement obtenir chaque face le même nombre de fois, c'est-à-dire 100 fois. Mais la réalité étant ce qu'elle, nous aurons une certaine distribution autour de 100, qui devrait être une distribution normale. Comment savoir si cette distribution des relevés autour de 100 est normale ou non ?

On se donne un chance de 5% de se tromper (c'est-à-dire qu'on est sûr de notre résultat à 95%). Nous faisons la sommes des carrés des différences des lectures réalisées avec le résultat théorique 100 et nous divisions chacun de ces résultats par le résultat théorique 100.

Le degré de liberté de notre échantillon est 5: chaque face à un nombre indépendant de chance d'avoir le résultat, mais la somme des résultats doit être 600. (le degré de liberté est toujours le degré de liberté moins un).

In [4]:
dice_face = np.arange(1,7)
dice_test = np.array([88, 109, 107, 94, 105, 97]) # résultat mesuré proposé
rigged_test = np.array([89,131,93,92,104,91])  # résultat mesuré pour un dé truqué

# dice_test = rigged_test # décommenter pour le dé truqué

nb_tests = 600
chance_per_face = 600 / 6
dice_label = np.full(6, chance_per_face) # le résultat théoriquement attendu

res = (np.square((dice_test-dice_label))/chance_per_face).sum()

confidence = 0.95
threshold = chi2.ppf(confidence, 5)

if res < threshold:
    print(f'Le dé n\'est pas truqué, {res:.2f} < {threshold:.2f}')
else:
    print(f'le dé est truqué car {res:.2f} ⩾ {threshold:.2f}')

Le dé n'est pas truqué, 3.44 < 11.07


## Vérifier si une distribution vérifie la [loi de Poisson](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html)

La [loi de Poisson](https://fr.wikipedia.org/wiki/Loi_de_Poisson) caractérise le comportement d'évènements se produisant durant un intervalle fixe, mais dont la fréquence ne dépend pas de cette durée.

La page de [Wikipédia](https://fr.wikipedia.org/wiki/Test_du_%CF%87%C2%B2) propose aussi un relevé par vérifier s'il suit une loi de Poisson. On observe un évènement qui semble intervenir avec une certain fréquence pendant une certaine durée. On a fait un échantillon de 100 valeurs, 31 évènements arrivent en moins de 1min, 45 en entre 1min et 2min (exclu), 16 entre 2min et 3min, 7 entre 3min et 4min et 1 entre 4min et plus. 

In [42]:
from tabulate import tabulate  # to print nice table

X = np.array([31,45,16,7,1]) # fréquence d'apparition des valeurs
print(tabulate(X.reshape(1,5), headers=np.arange(5), tablefmt='rounded_grid'))

╭─────┬─────┬─────┬─────┬─────╮
│   0 │   1 │   2 │   3 │   4 │
├─────┼─────┼─────┼─────┼─────┤
│  31 │  45 │  16 │   7 │   1 │
╰─────┴─────┴─────┴─────┴─────╯


On reprend l'échantillon on mettant dans la classe 'plus de 3min' l'échantillon dans 3min et 4min.

In [67]:
X1 = np.array([31,45,16,8]) # on a regroupé les deux dernières classes
print(tabulate(X1.reshape(1,4), headers=[0,1,2,3], tablefmt='rounded_grid'))

╭─────┬─────┬─────┬─────╮
│   0 │   1 │   2 │   3 │
├─────┼─────┼─────┼─────┤
│  31 │  45 │  16 │   8 │
╰─────┴─────┴─────┴─────╯


On calcule la fréquence moyenne d'arrivé de l'évènement.

In [78]:
mu = np.average(range(4), weights=X1)
mu = mu * X1.sum() / (X1.sum()-1)  # espérance empirique
# mu = np.dot(np.arange(4), X1) / (X1.sum() - 1)
print(f'La fréquence moyenne est {mu:.2f}')

La fréquence moyenne est 1.02


On calcule maintenant ce que donnerait une distribution de Poisson pour les trois premiers éléments.

In [80]:
from scipy.stats import poisson

table = np.array(range(3)).reshape(1,3)
y = np.around(poisson.pmf(range(3), mu)*100, 2)

print(tabulate(y.reshape(1,3), headers=[0,1,2], tablefmt='rounded_grid'))

╭───────┬───────┬───────╮
│     0 │     1 │     2 │
├───────┼───────┼───────┤
│ 36.05 │ 36.78 │ 18.76 │
╰───────┴───────┴───────╯


Pour le dernier élément, on prend l'inverse de la somme des probabilité de 0 à 3.

In [81]:
y = np.append(y, 100- poisson.cdf(2,mu)*100)

print(tabulate(y.reshape(1,4), headers=('0','1','2','≥ 3'), tablefmt='rounded_grid'))

╭───────┬───────┬───────┬─────────╮
│     0 │     1 │     2 │     ≥ 3 │
├───────┼───────┼───────┼─────────┤
│ 36.05 │ 36.78 │ 18.76 │ 8.40546 │
╰───────┴───────┴───────┴─────────╯


On fait maintenant le calcul des différences (au carré)

In [85]:
res = 0
for i,j in zip(X1, y):
    res += np.square(i-j) / j
print(f'res = {res:.2f}')

res = 2.97


In [90]:
# autre méthode d'écrire plus vite le résultat
res = (np.square(X1-y)/y).sum()
print(f'res = {res:.2f}')

res = 2.97


La variable $\chi^2$ est ici à deux degrés de liberté.

In [61]:
confidence = 0.95
freedom = 2    # ici deux degrés de liberté
threshold = chi2.ppf(confidence, freedom)  
print(f'Pour la distribution T, avec {freedom:.0f} degrés de liberté, P(T < {threshold:.2f}) = {confidence*100:.0f}%')

if res < threshold:
    print(f'La distribution suite la loi de Poisson, {res:.2f} < {threshold:.2f}')
else:
    print(f'La distribution suite la loi de Poisson {res:.2f} ⩾ {threshold:.2f}')

Pour la distribution T, avec 2 degrés de liberté, P(T < 5.99) = 95%
La distribution suite la loi de Poisson, 2.97 < 5.99
